In [3]:
import pandas as pd
from tqdm import tqdm
import time
import re
import requests
import difflib
import operator
from difflib import SequenceMatcher

In [155]:
import diff_match_patch as dmp_module
dmp_module.Match_Distance = 50
dmp = dmp_module.diff_match_patch()

In [144]:
def calc_ratio_t(a, b):
    diff = dmp.diff_main(a, b)
    dleven = dmp.diff_levenshtein(diff)
    return 1 - (dleven / max(len(aw), len(bw)))

In [145]:
def calc_match(text, pattern):
    loc = dmp.match_main(text, pattern, 0)
    print(loc)
    return loc >= 0

In [146]:
calc_ratio_t("BUTYROSPERMUM PRKII BUTTER asd", "BUTYROSPERMUM PARKII BUTTER")

0.8529411764705882

In [147]:
calc_match("asd BUTYROSPERMUM PARKII BUTTER", "BUTYROSPERMUM PRKII (Shea) BUTTER")

4


True

In [167]:
from fuzzywuzzy import fuzz 
fuzz.ratio("BUTYROSPERMUM PRKII (Shea) BUTTER", "BUTYROSPERMUM PARKII BUTTER") / 100

0.87

In [193]:


class FuzzyDict(dict):
    "Provides a dictionary that performs fuzzy lookup"
    def __init__(self, items = None, cutoff = .6):
        """Construct a new FuzzyDict instance

        items is an dictionary to copy items from (optional)
        cutoff is the match ratio below which mathes should not be considered
        cutoff needs to be a float between 0 and 1 (where zero is no match
        and 1 is a perfect match)"""
        super(FuzzyDict, self).__init__()

        if items:
            self.update(items)
        self.cutoff =  cutoff

        # short wrapper around some super (dict) methods
        self._dict_contains = lambda key: \
            super(FuzzyDict,self).__contains__(key)

        self._dict_getitem = lambda key: \
            super(FuzzyDict,self).__getitem__(key)

    def _search(self, lookfor, stop_on_first = False):
        """Returns the value whose key best matches lookfor

        if stop_on_first is True then the method returns as soon
        as it finds the first item
        """

        # if the item is in the dictionary then just return it
        if self._dict_contains(lookfor):
            return True, lookfor, self._dict_getitem(lookfor), 1

        # set up the fuzzy matching tool
        #ratio_calc = difflib.SequenceMatcher()
        #ratio_calc.set_seq1(lookfor)

        # test each key in the dictionary
        best_ratio = 0
        best_match = None
        best_key = None
        for key in self:

            # if the current key is not a string
            # then we just skip it
            if not isinstance(key, str):
                continue

            # we get an error here if the item to look for is not a
            # string - if it cannot be fuzzy matched and we are here
            # this it is defintely not in the dictionary
            try:
            # calculate the match value
                ratio = fuzz.ratio(lookfor, key)/100
            except TypeError:
                break

            # if this is the best ratio so far - save it and the value
            if ratio > best_ratio:
                best_ratio = ratio
                best_key = key
                best_match = self._dict_getitem(key)

            if stop_on_first and ratio >= self.cutoff:
                break

        return (
            best_ratio >= self.cutoff,
            best_key,
            best_match,
            best_ratio)


    def __contains__(self, item):
        "Overides Dictionary __contains__ to use fuzzy matching"
        if self._search(item, True)[0]:
            return True
        else:
            return False

    def __getitem__(self, lookfor):
        "Overides Dictionary __getitem__ to use fuzzy matching"
        matched, key, item, ratio = self._search(lookfor)

        if not matched:
            raise KeyError(
                "'%s'. closest match: '%s' with ratio %.3f"%
                    (str(lookfor), str(key), ratio))

        return item

In [5]:
def fuzzy_match_ingredients(ing_list, fuzdict):
    match_dict = {}
    for ing in tqdm(ing_list):
        if ing in match_dict.keys():
            continue
        upper_ing = ing.upper()
        if fuzdict.__contains__(upper_ing):
            match_dict[ing] = fuzdict[upper_ing]
    
    return match_dict

In [6]:
def cosing_lookup(ing_list, match_dict, desc, restrictions, functions):
    res = []
    for item in ing_list:
        key = match_dict.get(item, "")
        if key == "":
            continue
        description = desc.get(key, "unknown")
        restrictn = restrictions.get(key, "unknown")
        functn = functions.get(key, "unknown")
        res.extend([[key, description, restrictn, functn]])
    
    return res

In [7]:
def create_cosing_dict(ingredient_df, col_name):
    col_dict = {name.strip(): row[col_name] for (idx, row) in ingredient_df.iterrows() for name in row['INCI name'].split('/')} # .split('/')

    return col_dict

In [8]:
dfcosing = pd.read_csv('../Database/ingredient_cosing_37309.csv')
dfcosing.drop(columns=['Unnamed: 0'], inplace=True)
dfcosing

,COSING Ref No,INCI name,Chem/IUPAC Name / Description,Restriction,Function
0,94753.0,DISODIUM TETRAMETHYLHEXADECENYLCYSTEINE FORMYL...,Disodium Tetramethylhexadecenylcysteine Formyl...,NaN,SKIN PROTECTING
1,96229.0,ASTROCARYUM VULGARE SEED BUTTER,Astrocaryum Vulgare Seed Butter is the fat obt...,NaN,"SKIN CONDITIONING, SKIN CONDITIONING - EMOLLIENT"
2,89177.0,BARLEY SH-POLYPEPTIDE-17,Barley sh-Polypeptide-17 is a single chain rec...,NaN,"HAIR CONDITIONING, SKIN CONDITIONING"
3,98580.0,DAUCUS CAROTA SATIVA LEAF EXTRACT,Daucus Carota Sativa (Carrot) Leaf Extract is ...,NaN,SKIN CONDITIONING - MISCELLANEOUS
4,89078.0,GOSSYPIUM HIRSUTUM SEED EXTRACT,Gossypium Hirsutum Seed Extract is the extrac...,NaN,SKIN CONDITIONING
...,...,...,...,...,...
37304,90014.0,PHELLODENDRON AMURENSE BARK,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"
37305,90014.0,LONICERA JAPONICA FLOWER,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"
37306,90014.0,CHAENOMELES SINENSIS FRUIT,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"
37307,90014.0,CAMELLIA SINENSIS LEAF,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"


In [199]:
# dfi3 = pd.read_csv('Database/ingredient_inci_1570.csv')
ingnames_dict = {name.strip(): name.strip() for name in dfcosing['INCI name']}
# print(ingnames_dict)
fd = FuzzyDict(ingnames_dict, cutoff = .5)
print(len(fd))

29908


In [195]:
len(ingnames_dict)

29908

In [196]:
# SOME TESTING SHIT
fuzzy_match_ingredients(["BUTYROSPERMUM (Shea) BUTTER"],fd)

100%|██████████| 1/1 [00:00<00:00,  7.76it/s]


{'BUTYROSPERMUM (Shea) BUTTER': 'BUTYROSPERMUM PARKII BUTTER'}

In [11]:
# ingnames_dict = {name.strip(): name.strip() for name in df_ingredient['INCI name']}
# fd = FuzzyDict(ingnames_dict, cutoff = .55)

# # Compare product ingredient list and database
# # match_dict = find_matching_ingredient(ing_list, rating, 0.55)
# match_dict = fuzzy_match_ingredients(ing_list, fd)
# print(match_dict)

## **Load EWG ingredient list**

In [12]:
def clean_string_name(string):
    text = string.replace(' Size: 4 oz. * USDA Certified Organic Ingredient ** None remains after saponifying oils into soap and glycerin', '')
    pattern = "[\|\*\_\'\{}&]".format('"')
    regex1 = re.compile('\[.*]')
    regex2 = re.compile('\[.*}')
    text = re.sub(pattern, "", text)
    text = re.sub(regex1, "", text)
    text = re.sub(regex2, "", text)
    text = text.strip('\x0c')
    text = text.strip(' ')
    text = text.upper()
    
    return text

In [13]:
def clean_string(string):
    pattern = "[\|\*\_\'\{}&]".format('"')
    regex = re.compile('\\\S*')
    text = re.sub(pattern, "", string)
    text = re.sub(",, ", ", ", text)
    text = re.sub(regex, " ", text)
    text = text.split(':')[1]
#     text = text.split('.')[0]
    text = text.strip()
#     split = [x.strip(' ').lower() for x in text.split(',')]

    return text

def string_to_list(text):
    pattern = "[\|\*\_\'\{}]".format('"')
    text = re.sub(pattern, "", text)
    text = text.replace('Aqua/Water/Eau', 'Aqua')
#     split = [x.strip(' ').lower() for x in text.split([\\.,]\\s)split('.')]
    split = [x.strip(' ').lower().replace('aqua/water/eau', 'aqua') for x in re.split("[,.]", text)]
    return split

In [40]:
df_skin = pd.read_csv('./ewg_makeup_full_10529.csv')

In [41]:
example = df_skin[df_skin['ingredient_list'] != 'No ingredient list']['ingredient_list'].to_list()

In [42]:
sample = [name.upper().strip() for name in example[4747].split(',')]
sample

['WATER/EAU',
 'STYRENE/ACRYLATES COPOLYMER',
 'PROPYLENE GLYCOL',
 'LAURETH-21',
 'PENTYLENE GLYCOL',
 'PEG-40 HYDROGENATED CASTOR OIL',
 'PHENOXYETHANOL',
 'AMMONIUM ACRYLATES COPOLYMER',
 'CAPRYLYL GLYCOL',
 'PPG-2-DECETH-30',
 'SODIUM DEHYDROACETATE',
 'SODIUM LAURYL SULFATE',
 'DISODIUM LAURETH SULFOSUCCINATE',
 '[+/- (MAY CONTAIN): BLACK 2/CI 77266].']

In [43]:
# match_dict = find_matching_ingredient(ing_list, rating, 0.55)
match_dict = fuzzy_match_ingredients(sample, fd)
print(match_dict)

100%|██████████| 14/14 [00:01<00:00, 10.84it/s]

{'WATER/EAU': 'SEA WATER', 'STYRENE/ACRYLATES COPOLYMER': 'STEARYL ACRYLATE COPOLYMER', 'PROPYLENE GLYCOL': 'PROPYLENE GLYCOL', 'LAURETH-21': 'LAURETH-21', 'PENTYLENE GLYCOL': 'PENTYLENE GLYCOL', 'PEG-40 HYDROGENATED CASTOR OIL': 'PEG-40 HYDROGENATED CASTOR OIL', 'PHENOXYETHANOL': 'PHENOXYETHANOL', 'AMMONIUM ACRYLATES COPOLYMER': 'AMMONIUM ACRYLATES COPOLYMER', 'CAPRYLYL GLYCOL': 'CAPRYLYL GLYCOL', 'PPG-2-DECETH-30': 'PPG-2-DECETH-30', 'SODIUM DEHYDROACETATE': 'SODIUM DEHYDROACETATE', 'SODIUM LAURYL SULFATE': 'SODIUM LAURYL SULFATE', 'DISODIUM LAURETH SULFOSUCCINATE': 'DISODIUM LAURETH SULFOSUCCINATE', '[+/- (MAY CONTAIN): BLACK 2/CI 77266].': 'C16-24 ALKYL C16-24 ACIDATE'}


In [44]:
for o in match_dict.values():
    print(o)

SEA WATER
STEARYL ACRYLATE COPOLYMER
PROPYLENE GLYCOL
LAURETH-21
PENTYLENE GLYCOL
PEG-40 HYDROGENATED CASTOR OIL
PHENOXYETHANOL
AMMONIUM ACRYLATES COPOLYMER
CAPRYLYL GLYCOL
PPG-2-DECETH-30
SODIUM DEHYDROACETATE
SODIUM LAURYL SULFATE
DISODIUM LAURETH SULFOSUCCINATE
C16-24 ALKYL C16-24 ACIDATE


In [45]:
df_skin

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list
0,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/929942-C...,C'est Moi,"Dream Beams Cream Highlighter, Aura",verified,https://static.ewg.org/skindeep_images/9299/92...,"Aqua/Water/Eau, Ethyl Macadamiate, Butyrosperm..."
1,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704637-M...,Mineral Fusion,"Bronzer, Luster Bronzer duo",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste..."
2,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/929943-C...,C'est Moi,"Dream Beams Cream Highlighter, Moonstruck",verified,https://static.ewg.org/skindeep_images/9299/92...,"Aqua/Water/Eau, Ethyl Macadamiate, Mica, Butyr..."
3,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704641-M...,Mineral Fusion,"Blush/Bronzer Duo, Rio Blonzer",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste..."
4,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/820241-B...,Beautycounter,"Illuminating Cream Highlighter, Bronze Glow",verified,https://static.ewg.org/skindeep_images/8202/82...,"Caprylic/Capric Triglyceride, Jojoba Esters, S..."
...,...,...,...,...,...,...,...,...
10524,Makeup,Glitter,https://www.ewg.org/skindeep/products/888847-B...,BH Cosmetics,"Glitter Collection, Black",05,https://static.ewg.org/skindeep_images/8888/88...,"Polyethylene Terephthalate, Acrylates Copolyme..."
10525,Makeup,Glitter,https://www.ewg.org/skindeep/products/879942-E...,e.l.f.,"Stardust Glitter, Cosmic Silver 74605",06,https://static.ewg.org/skindeep_images/8799/87...,"POLYETHYLENE TEREPHTHALATE, POLYSILICON-1, ALU..."
10526,Makeup,Glitter,https://www.ewg.org/skindeep/products/884475-L...,L.A. Colors,"Glitter Palette, Cgp694 Delightful",06,https://static.ewg.org/skindeep_images/8844/88...,"MINERAL OIL (PARAFFINUM LIQUIDUM), POLYISOBUTE..."
10527,Makeup,Glitter,https://www.ewg.org/skindeep/products/934403-W...,Wet N Wild,"Glitter Liner, Turnt Up 1230167",06,https://static.ewg.org/skindeep_images/9344/93...,"WATER, POLYETHYLENE TEREPHTHALATE, GLYCERIN, V..."


In [46]:
df_skin[df_skin['ingredient_list'] == 'No ingredient list']

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list
7173,Makeup,Lip Gloss,https://www.ewg.org/skindeep/products/920738-S...,Saint,"Amplified Lip Lacquer, Barolo",01,https://static.ewg.org/skindeep_images/9207/92...,No ingredient list
7432,Makeup,Lip Gloss,https://www.ewg.org/skindeep/products/938211-U...,Uoma,"Beauty Boss Gloss Liquid Marble, Ambition",04,https://static.ewg.org/skindeep_images/9382/93...,No ingredient list


In [53]:
drop_index = df_skin[df_skin['ingredient_list'] == 'No ingredient list'].index.to_list()
df_skin.drop(index=drop_index, inplace=True)
df_skin.count()

cat_name           10527
subcat_name        10527
product_url        10527
product_brand       9715
product_name       10527
product_score      10527
product_img        10527
ingredient_list    10527
dtype: int64

In [54]:
def compare_ingredient(x):
    x = clean_string_name(x)
    sample = [name.strip() for name in x.split(',')]
    match_dict = fuzzy_match_ingredients(sample, fd)
    res = [i for i in match_dict.values()]
#     print(sample)
#     print(res)
    
    return res

In [55]:
df_skin.to_csv('ewg_makeup_full_after_processing_10527.csv', index=False)

In [ ]:
df_testing = df_skin.iloc[:130, :140]
df_testing

In [200]:
df_testing['new_ing_list'] = df_testing['ingredient_list'].apply(compare_ingredient)

100%|██████████| 15/15 [00:00<00:00, 22.42it/s]
/Users/cedric/miniconda3/envs/cs_ftmle/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [202]:
df_skin['new_ing_list'] = df_skin['ingredient_list'].apply(compare_ingredient)

100%|██████████| 10/10 [00:00<00:00, 57.12it/s]


100%|██████████| 15/15 [00:00<00:00, 31.04it/s]


100%|██████████| 14/14 [00:00<00:00, 16.89it/s]


100%|██████████| 11/11 [00:00<00:00, 32.01it/s]


100%|██████████| 22/22 [00:00<00:00, 46.82it/s]


...

100%|██████████| 47/47 [00:02<00:00, 23.33it/s]


100%|██████████| 43/43 [00:01<00:00, 31.55it/s] 


100%|██████████| 55/55 [00:02<00:00, 19.15it/s]


100%|██████████| 28/28 [00:01<00:00, 25.19it/s]


100%|██████████| 6/6 [00:00<00:00, 18.45it/s]


In [203]:
df_skin.to_csv('ewg_makeup_done_processing_ced.csv', index=False)

In [77]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [201]:
df_testing[['ingredient_list', 'new_ing_list']]

,ingredient_list,new_ing_list
0,"Aqua/Water/Eau, Ethyl Macadamiate, Butyrospermum Parkii (Shea) Butter*, Rhus Verniciflua Peel Wax, Mica, Titanium Dioxide (CI 77891), Glycerin*, Potassium Cetyl Phosphate, Cetearyl Olivate, Sorbitan Olivate, Xylitylglucoside, Hydroxyacetophenone, Prunus Armeniaca (Apricot) Kernel Oil*, Anhydroxylitol, Carbomer, 1,2-Hexanediol, Caprylyl Glycol, Xylitol, Sodium Phytate, Tocopherol*, Aloe Barbadensis Leaf Juice*, Helianthus Annuus (Sunflower) Seed Oil*, Arginine, Citric Acid, Iron Oxides (CI 77491, CI 77499).","[CAVIAR WATER, ETHYL MACADAMIATE, BUTYROSPERMUM PARKII BUTTER, RHUS VERNICIFLUA PEEL WAX, MICA, TITANIUM DIOXIDE, GLYCERIN, POTASSIUM CETYL PHOSPHATE, CETEARYL OLIVATE, SORBITAN OLIVATE, XYLITYLGLUCOSIDE, HYDROXYACETOPHENONE, PRUNUS ARMENIACA KERNEL OIL, ANHYDROXYLITOL, CARBOMER, 12, 1,2-HEXANEDIOL, CAPRYLYL GLYCOL, XYLITOL, SODIUM PHYTATE, TOCOPHEROL, ALOE BARBADENSIS LEAF JUICE, HELIANTHUS ANNUUS SEED OIL, ARGININE, CITRIC ACID, IRON OXIDES, CI 77499]"
1,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Stearate, Lauroyl Lysine, Caprylic/Capric Triglyceride, Tocopheryl Acetate (Vitamin E), Ascorbyl Palmitate (Vitamin C), Camellia Sinensis (White Tea) Leaf Extract, Aspalathus Linearis (Rooibos Red Tea) Leaf Extract, Punica Granatum (Pomegranate) Extract, Laminaria Ochroleuca (Sea Kelp) Extract, Aloe Barbadensis (Aloe Vera) Leaf Extract, Sambucus Nigra (Elderberry) Fruit Extract, Phenoxyethanol, May contain: Iron Oxides, Titanium Dioxide","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC STEARATE, LAUROYL LYSINE, CAPRIC TRIGLYCERIDE, TOCOPHERYL ACETATE, ASCORBYL PALMITATE, CAMELLIA SINENSIS LEAF EXTRACT, ASPALATHUS LINEARIS LEAF EXTRACT, PUNICA GRANATUM PEEL EXTRACT, LAMINARIA OCHROLEUCA EXTRACT, ALOE BARBADENSIS LEAF EXTRACT, SAMBUCUS NIGRA FRUIT EXTRACT, PHENOXYETHANOL, IRON OXIDES, TITANIUM DIOXIDE]"
2,"Aqua/Water/Eau, Ethyl Macadamiate, Mica, Butyrospermum Parkii (Shea) Butter*, Titanium Dioxide (CI 77891), Rhus Verniciflua Peel Wax, Glycerin*, Potassium Cetyl Phosphate, Synthetic Fluorphlogopite, Cetearyl Olivate, Sorbitan Olivate, Xylitylglucoside, Hydroxyacetophenone, Prunus Armeniaca (Apricot) Kernel Oil*, Anhydroxylitol, Carbomer, 1,2-Hexanediol, Caprylyl Glycol, Xylitol, Sodium Phytate, Tocopherol*, Tin Oxide, Aloe Barbadensis Leaf Juice*, Helianthus Annuus (Sunflower) Seed Oil*, Arginine, Citric Acid.","[CAVIAR WATER, ETHYL MACADAMIATE, MICA, BUTYROSPERMUM PARKII BUTTER, TITANIUM DIOXIDE, RHUS VERNICIFLUA PEEL WAX, GLYCERIN, POTASSIUM CETYL PHOSPHATE, SYNTHETIC FLUORPHLOGOPITE, CETEARYL OLIVATE, SORBITAN OLIVATE, XYLITYLGLUCOSIDE, HYDROXYACETOPHENONE, PRUNUS ARMENIACA KERNEL OIL, ANHYDROXYLITOL, CARBOMER, 12, 1,2-HEXANEDIOL, CAPRYLYL GLYCOL, XYLITOL, SODIUM PHYTATE, TOCOPHEROL, TIN OXIDE, ALOE BARBADENSIS LEAF JUICE, HELIANTHUS ANNUUS SEED OIL, ARGININE, CITRIC ACID]"
3,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Stearate, Lauroyl Lysine, Caprylic/Capric Triglyceride, Tocopheryl Acetate (Vitamin E), Ascorbyl Palmitate (Vitamin C), Camellia Sinensis (White Tea) Leaf Extract, Aspalathus Linearis (Rooibos Red Tea) Leaf Extract, Punica Granatum (Pomegranate) Extract, Laminaria Ochroleuca (Sea Kelp) Extract, Aloe Barbadensis (Aloe Vera) Leaf Extract, Sambucus Nigra (Elderberry) Fruit Extract, Phenoxyethanol, May contain: Calcium Sodium Borosilicate, Carmine, Iron Oxides, Manganese Violet, Tin Oxide, Titanium Dioxide","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC STEARATE, LAUROYL LYSINE, CAPRIC TRIGLYCERIDE, TOCOPHERYL ACETATE, ASCORBYL PALMITATE, CAMELLIA SINENSIS LEAF EXTRACT, ASPALATHUS LINEARIS LEAF EXTRACT, PUNICA GRANATUM PEEL EXTRACT, LAMINARIA OCHROLEUCA EXTRACT, ALOE BARBADENSIS LEAF EXTRACT, SAMBUCUS NIGRA FRUIT EXTRACT, PHENOXYETHANOL, CALCIUM SODIUM BOROSILICATE, CALAMINE, IRON OXIDES, MANGANESE DIOXIDE, TIN OXIDE, TITANIUM DIOXIDE]"
4,"Caprylic/Capric Triglyceride, Jojoba Esters, Silica, Butyrospermum Parkii (Shea) Butter, Euphorbia Cerifera (Candelilla) Wax/Cire de candelilla, Mica,

In [75]:
df_testing

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list,new_ing_list
0,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/929942-C...,C'est Moi,"Dream Beams Cream Highlighter, Aura",verified,https://static.ewg.org/skindeep_images/9299/92...,"Aqua/Water/Eau, Ethyl Macadamiate, Butyrosperm...","[TEA-LAURATE, ETHYL MACADAMIATE, BUTYROSPERMUM..."
1,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704637-M...,Mineral Fusion,"Bronzer, Luster Bronzer duo",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
2,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/929943-C...,C'est Moi,"Dream Beams Cream Highlighter, Moonstruck",verified,https://static.ewg.org/skindeep_images/9299/92...,"Aqua/Water/Eau, Ethyl Macadamiate, Mica, Butyr...","[TEA-LAURATE, ETHYL MACADAMIATE, MICA, BUTYROS..."
3,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704641-M...,Mineral Fusion,"Blush/Bronzer Duo, Rio Blonzer",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
4,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/820241-B...,Beautycounter,"Illuminating Cream Highlighter, Bronze Glow",verified,https://static.ewg.org/skindeep_images/8202/82...,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, B..."
5,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704639-M...,Mineral Fusion,"Bronzer, Sparkle Bronzer",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
6,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/646346-R...,Rejuva Minerals,"Bronzer, Caribbean",verified,https://static.ewg.org/skindeep_images/6463/64...,"Boron Nitride,Â Kaolin, Â *CoperniciaÂ Cerifer...","[BORON NITRIDE, KAOLIN, COPERNICIA CERIFERA WA..."
7,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/820244-B...,Beautycounter,"Illuminating Cream Highlighter, Rose Glow",verified,https://static.ewg.org/skindeep_images/8202/82...,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, M..."
8,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/820242-B...,Beautycounter,"Illuminating Cream Highlighter, Pearl Glow",verified,https://static.ewg.org/skindeep_images/8202/82...,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, C..."
9,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/782243-W...,W3LL PEOPLE,Bio Bronzer Baked Powder,verified,https://static.ewg.org/skindeep_images/7822/78...,"""MICA, ZEA MAYS STARCH / ZEA MAYS (CORN) STARC...","[MICA, ARACHIS HYPOGAEA (PEANUT) SPROUT EXTRA..."


## **SequenceMatcher**

In [83]:
def find_matching_ingredient(my_ingredients, rating_dict, thresh=0.5):
    ''' my_ingredients: list of product's ingredients
        Loop thru each ingredient in the ingredient list of the products
        then check if that ingredient appears in our ingredient list
        Calculate match_metric using SequenceMatcher and return the highest score and the best match
        Compare the match_metric with thresh > append to match_dict
    '''
    match_dict = {}
    for ingredient in tqdm(my_ingredients):
        if ingredient in match_dict.keys():
            continue
        match_metric = {key: SequenceMatcher(None, key, ingredient).ratio() for key in rating_dict.keys()}
        best_match, best_metric = max(match_metric.items(), key=operator.itemgetter(1)) # What is operator.itemgetter(1)
        if best_metric > thresh:
            match_dict[ingredient] = best_match
        else:
            match_dict[ingredient] = 'unknown'
    return match_dict

In [89]:
def compare_ingredient(x):
    x = clean_string_name(x)
    sample = [name.strip() for name in x.split(',')]
    match_dict = find_matching_ingredient(sample, fd)
    res = [i for i in match_dict.values()]
#     print(sample)
#     print(res)
    
    return res

In [90]:
df_testing['new_ing_list'] = df_testing['ingredient_list'].apply(compare_ingredient)

100%|██████████| 9/9 [00:21<00:00,  2.38s/it]
2020-10-17 20:28:07,599 [16154] WARNING  py.warnings:110: [JupyterRequire] /Users/macbookpro/miniconda3/envs/cs_ftmle/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.



In [91]:
df_skin['product_score'].value_counts()

04          1561
01          1347
03          1225
05          1222
02          1203
verified    1132
06           369
07           102
08            36
09            10
10             1
Name: product_score, dtype: int64